In [1]:
class TargetArea:
    
    def __init__(self, x1: int, x2: int, y1: int, y2: int):
        assert x1 < x2
        assert y1 < 0 and y2 < 0
        assert y1 > y2
        
        self.x1 = x1
        self.x2 = x2
        self.y1 = y1
        self.y2 = y2
    
    @classmethod
    def from_raw(cls, raw_target: str):
        raw_xs, raw_ys = raw_target.replace("target area: ", "").split(", ")
        raw_x1, raw_x2 = raw_xs[2:].split("..")
        raw_y2, raw_y1 = raw_ys[2:].split("..")
        
        return cls(int(raw_x1), int(raw_x2), int(raw_y1), int(raw_y2))
    
    def __str__(self):
        return f"target area: x={self.x1}..{self.x2}, y={self.y2}..{self.y1}"
    
    def includes(self, x: int, y:int) -> bool:
        return (self.x1 <= x <= self.x2) and (self.y1 >= y >= self.y2)
    
    def is_missed_by(self, x: int, y: int) -> bool:
        """
        Returns True is (x, y) have gone beyond where the target area is:
        - x is to the right of the target area
        OR
        - y is below the target area
        """
        return (x > self.x2) or (y < self.y2)
        

class Probe:
    
    def __init__(self, x_vel: int, y_vel: int, target_area: TargetArea, debug=False):
        self.x = 0
        self.y = 0
        self.x_vel = x_vel
        self.y_vel = y_vel
        
        self.max_y = self.y
        
        self.target_area = target_area
        self.hits_target = self.target_area.includes(self.x, self.y)
        
        self.num_steps = 0
        
        self.debug = debug
        if self.debug:
            print(f"VELOCITY: {x_vel}, {y_vel}")
            print(f"Step   0: ({self.x:4},{self.y:4})")
        
        self.launch()
        
    def launch(self):
        """
        Keeps stepping until we've reached or passed the target area.
        """
        breaker = 0
        while self.keep_stepping():
            breaker += 1
            if breaker > 10000:
                # Just in case I messed up, prevent infinite loop
                break
            
            self.step()
        
    def keep_stepping(self):
        in_target = self.target_area.includes(self.x, self.y)
        beyond_target = self.target_area.is_missed_by(self.x, self.y)
        return not in_target and not beyond_target
        
    def step(self):
        self.x += self.x_vel
        self.y += self.y_vel
        self.max_y = max(self.max_y, self.y)
        
        if self.x_vel > 0:
            self.x_vel -= 1
        elif self.x_vel < 0:
            self.x_vel += 1
            
        self.y_vel -= 1
        
        in_target = self.target_area.includes(self.x, self.y)
        if in_target:
            self.hits_target = True
        
        self.num_steps += 1
        
        if self.debug:
            print(f"Step {self.num_steps:3}: ({self.x:4},{self.y:4}) "
                  f"{'IN_TARGET' if in_target else ''}")

In [2]:
def find_answers(target_area: TargetArea):
    best_height = -1
    num_distinct = 0
    
    n = 0
    while n*(n+1)/2 < target_area.x1:
        n += 1
    
    min_x_vel = n
    max_x_vel = target_area.x2
    
    min_y_vel = target_area.y2
    max_y_vel = abs(target_area.y2) - 1
    
    for x_vel in range(min_x_vel, max_x_vel+1):
        for y_vel in range(min_y_vel, max_y_vel+1):
            probe = Probe(x_vel, y_vel, target_area)
            
            if probe.hits_target:
                num_distinct += 1
                
                if probe.max_y > best_height:
                    best_height = probe.max_y
                    best_vels = (x_vel, y_vel)
                
    return best_height, num_distinct

In [3]:
# Test Case
test_target_area = TargetArea.from_raw('target area: x=20..30, y=-10..-5')

best_height, num_distinct = find_answers(test_target_area)
assert best_height == 45
assert num_distinct == 112

In [4]:
target_area = TargetArea.from_raw('target area: x=206..250, y=-105..-57')

best_height, num_distinct = find_answers(target_area)

print("Part 1 answer: Best height is", best_height)
print("Part 2 answer: Number of distinct initial velocity values is", num_distinct)

Part 1 answer: Best height is 5460
Part 2 answer: Number of distinct initial velocity values is 3618
